# Deep learning for text: sequence models
With the improvement using the bigram data we have seen that the order of words matters. Now we want to study more in depth how to exploit the order of words in texts to perform several tasks that are common when using textual data. Instead of creating N-grams we will let the model figure out how to use a text to address the task at hand using a sequnce model. We start by assigning an integer index to each word in a vocabulary that will be mapped to a vector. These vectors represent a base vectors that will be used to represent every text. 

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_length,
)

In [ ]:
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)